# 数据观察和预处理

我们的数据放在 `BUPT-ML-FinalProject\release\data` 中，其中 `train.csv` 为训练集，`test.csv` 为测试集，`sample.csv` 为提交样例

## 数据观察

1. 直观上看，数据量很小，`train.csv` 和 `test.csv` 分别只有 981 和 979 条样本
2. 训练集的格式为 `sentiment,text`；测试集的格式为 `id,text`
3. 官方没有给 `val.csv`，这意味着我们需要按比例随机划分出验证集
4. 训练集中情感 1-5 的比例并不均衡，直观上大致符合中性情感最多，极端情感较少的规律，需要进一步根据统计情况分析
5. 数据中包含一些需要清洗掉的部分，比如
    - url 链接
    - `@` 引导的用户名
    - `""` 双引号
    - `Ì¢‰âÂ‰ã¢` 等乱码
    - `&amp;` 在 HTML 中表示 `&`，考虑暂时替换成单词 `and`（或许可能还有其它 HTML 编码）
    - `#xxx` 表示话题，包含有信息，不能直接删除，只能删掉符号

## 预处理

In [7]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
import os

In [8]:
cols_train = ['sentiment','text']
cols_test = ['id','text']
df_train = pd.read_csv('./data/release/train.csv', header=0)
df_test = pd.read_csv('./data/release/test.csv', header=0)

In [9]:
def data_cleaning(text):
    text = re.sub(r'[^a-z_A-Z0-9-\.!@#\$%\\\^&\*\)\(\+=\{\}\[\]\/\"\,\'<>~\·`\?:;|\s]', '', str(text))  # 非字母、数字、符号
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)  # url
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)  # @
    text = text.replace('"', '').replace('#', '').replace('&', 'and')  # "" and # and &
    text = BeautifulSoup(text, 'lxml').get_text()  # html编码
    text = re.sub(r'[_\-@#\$\^\*\)\(\+=\{\}\[\]\/\"<>~`:;|]', '', text)  # 剩下的一些无用符号
    return text

In [10]:
df_train['text'] = df_train['text'].map(data_cleaning)
df_test['text'] = df_test['text'].map(data_cleaning)
# label minus one to fit the model
# df_train['sentiment'] = df_train['sentiment'].map(lambda x: x-1)

d:\programs\Anaconda3\envs\pytorch\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [11]:
df_test.to_csv('./data/preprocessed/test.csv', index=False)

headers = pd.DataFrame(columns=cols_train)
headers.to_csv('./data/preprocessed/train.csv', index=False)
headers.to_csv('./data/preprocessed/dev.csv', index=False)

val_ratio = 0.2
split_seed = 1234
for i in range(1, 6):
    df_train_split, df_val_split = train_test_split(df_train[df_train['sentiment']==i], test_size=val_ratio, random_state=split_seed)
    df_train_split.to_csv('./data/preprocessed/train.csv', index=False, mode='a', header=None)
    df_val_split.to_csv('./data/preprocessed/dev.csv', index=False, mode='a', header=None)
